# Bokeh graphs
Using the Bokeh package, interactive graphs can be produced. At first, the data is copied to a datasource and Bokeh is initialized ([Data preparation](#Data-preparation)). For the particle analysis, a number of graphs are prepared:

- [Particle distribution](#Particle-distribution): Distribution of the particles over the substrate
- [Particle morphology](#Particle-morphology): Plot of the particle diameter against the aspect ratio
- [Chemical composition](#Chemical-composition): Plot containing two elements quantified during the EDX measurement
- [Particle size distribution](#Particle-size-distribution): Histogram of the particle size
- [Inter-particle distance](#Inter-particle-distance): Histogram of the inter-particle distance

In order to couple different graphs, it seem to be required to produce them in a single cell. Therefore, all graphs are prepared as functions which is finally called in the [last cell](#Output-graphs).

#### Preparation
As this script is designed to be used both externally and stand-alone, at first some required modules are loaded. Then, some global variables are checked whether they have been created externally and if not, are assigned.

In [3]:
# Import modules
import numpy as np
import pandas as pd
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.layouts import gridplot

In [21]:
# DISABLE_BOKEH
try:
    DISABLE_BOKEH
except NameError:
    DISABLE_BOKEH = False
    print("DISABLE_BOKEH not specified, set to "+str(DISABLE_BOKEH))
    
# directory
try:
    directory
except NameError:
    directory = "F:\\PA_UC\\"
    print("Directory not specified, set to "+directory)

# stub
try:
    stub
except NameError:
    stub = 1
    print("Stub not specified, set to "+str(stub))
    
# data
try:
    data
except NameError:
    print("No data available, running ImportData:")
    %run ./ImportData.ipynb
    print("-----")
    
# Interparticle distance
try:
    data["Dist"]
except KeyError:
    print("No data on the inter-particle distance available, running InterparticleDistance:")
    %run ./InterparticleDistance.ipynb
    print("-----")

No data on the inter-particle distance available, running InterparticleDistance:
dist_required not specified, set to 75
Median interparticle distance: 9.0 um
Fraction of particles further than 75 um apart: 0.0%
-----


#### Data preparation
In order to make the graphs interactive, a ColumnDataSource needs to be defined containing the data to be plotted. For simplicity, the whole *data* dataframe is copied to the Datasource *source*. Also, set Bokeh to output the graphs in the Jupyter notebook directly.

In [4]:
# Prepare Bokeh
output_notebook()

# Set datasource
source = ColumnDataSource(data)

Loading BokehJS ...

#### Particle distribution
A plot is prepared of the particle distribution over the substrate.

In [6]:
def BokehPlotDistribution():
    pDist = figure(
        tools="crosshair,pan,wheel_zoom,box_zoom,reset,box_select,lasso_select,tap,save,hover", 
        active_drag="lasso_select", 
        active_scroll="wheel_zoom", 
        title="Distribution", 
        x_axis_label="Horizontal position x / mm", 
        y_axis_label="Vertical position y / mm")

    pDist.circle(
        'X', 
        'Y', 
        source=source, 
        size=3, 
        line_color='#005B82', 
        fill_color='#005B82', 
        fill_alpha=1
    )
    
    return pDist

#### Particle morphology
A plot of the aspect ratio against the mean particle diameter.

In [7]:
def BokehPlotMorphology():
    pDA = figure(
        tools="crosshair,pan,wheel_zoom,box_zoom,reset,box_select,lasso_select,tap,save,hover", 
        active_drag="lasso_select", 
        active_scroll="wheel_zoom", 
        title="Morphology", 
        x_axis_label="Mean particle diameter / um", 
        y_axis_label="Aspect ratio")

    pDA.circle(
        'd', 
        'A', 
        source=source, 
        size=3, 
        line_color="#005B82", 
        fill_color="#005B82", 
        fill_alpha=1
    )
    
    return pDA

#### Chemical composition
A plot showing two elements (weight fraction). The function is prepared with two arguments, which can be used to set which elements should be plotted. The column names should correspond to the column names.

In [10]:
def BokehPlotChemistry(chX='UM', chY='OK'):
    # Check first if columns exists
    if not chX in data.columns:
        print("X column ("+chX+") not found! UM used instead.")
        chX = "UM"
    
    if not chY in data.columns:
        print("Y column ("+chY+") not found! OK used instead.")
        chY = "OK"
        
    pChem = figure(
        tools="crosshair,pan,wheel_zoom,box_zoom,reset,box_select,lasso_select,tap,save,hover", 
        active_drag="lasso_select", 
        active_scroll="wheel_zoom", 
        title="Chemical composition", 
        x_axis_label="Weight fraction "+chX, 
        y_axis_label="Weight fraction "+chY)

    pChem.circle(
        chX, 
        chY, 
        source=source, 
        size=3, 
        line_color='#005B82', 
        fill_color='#005B82', 
        fill_alpha=1
    )
    return pChem

#### Particle size distribution
A histogram of the particle size distribution is shown. The histogram is not interactive. The function contains a single argument to set the total number of bins.

In [15]:
def BokehPlotHistogram(bins=50):
    # Prepare data
    hist, edges = np.histogram(data["d"], density=True, bins=bins)

    # Graph
    pHist = figure(
        tools="crosshair,pan,wheel_zoom,box_zoom,reset,box_select,lasso_select,tap,save,hover", 
        active_drag="lasso_select", 
        active_scroll="wheel_zoom", 
        title="Particle size distribution", 
        x_axis_label="Particle diameter d / um", 
        y_axis_label="Number of particles")

    pHist.quad(
        top=hist, 
        bottom=0, 
        left=edges[:-1], 
        right=edges[1:],
        fill_color="#005B82", 
        fill_alpha=0.25,
        line_color="#005B82"
    )
    
    pHist.line(
        [np.median(data["d"]), np.median(data["d"])],
        [0, max(hist)],
        line_width=1,
        line_color="red"
    )

    return pHist

#### Inter-particle distance
Plotting a histogram of the interparticle distance. The graphs is not interacxtive. The function contains a single argument, which can be used to set the number of bins.

In [23]:
def BokehPlotDistance(bins=50):
    # Prepare data
    histNN, edgesNN = np.histogram(data["Dist"], density=True, bins=bins)

    # Graph
    pNN = figure(
        tools="crosshair,pan,wheel_zoom,box_zoom,reset,box_select,lasso_select,tap,save,hover", 
        active_drag="lasso_select", 
        active_scroll="wheel_zoom", 
        title="Inter-particle distance", 
        x_axis_label="Distance to neirest neighbor / um", 
        y_axis_label="Number of particles")

    pNN.quad(
        top=histNN, 
        bottom=0, 
        left=edgesNN[:-1], 
        right=edgesNN[1:],
        fill_color="#005B82", 
        fill_alpha=0.25,
        line_color="#005B82"
    )
    
    #xx = np.median(data["Dist"])
    xx = 75
    pNN.line(
        [xx, xx],
        [0, max(histNN)],
        line_width=1,
        line_color="red"
    )

    return pNN

#### Output graphs
This cell calls all defined functions and finally shows the graphs.

In [22]:
# Prepare a grid with the graphs
pAll = gridplot(
    [
        [BokehPlotDistribution(), BokehPlotMorphology()], 
        [BokehPlotChemistry()],
        [BokehPlotHistogram(), BokehPlotDistance()]
    ],
    plot_width=490,
    plot_height=490
)

# Finally, plot the graphs (only when enabled)
if(DISABLE_BOKEH==False):
    fig = show(pAll, notebook_handle=True)